In [1]:
import autogen
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

from pathlib import Path
import os

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

code_executor = DockerCommandLineCodeExecutor(timeout=60, work_dir=work_dir)

config_list = [
    {
        'model': os.getenv("OPENAI_DEFAULT_MODEL"),
        'api_key': os.getenv("OPENAI_API_KEY")
    }
]

llm_config = {
    "seed": 42,  
    "temperature": 0.7, 
    "config_list": config_list,
    "timeout": 600,  
}

assistant_config = {
    **llm_config,
}

In [2]:
assistant = autogen.AssistantAgent(
    name="Python_Expert",
    llm_config=assistant_config,
    system_message="""당신은 전문 Python 프로그래머입니다.
    코드를 작성할 때는 다음 원칙을 따릅니다:
    1. 깨끗하고 효율적인 코드 작성
    2. 적절한 주석 추가
    3. 에러 처리 포함
    4. 모듈화된 구조 사용
    작성한 코드는 실행하여 결과를 확인합니다."""
)

In [3]:
user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=5,
    code_execution_config={
        "executor": code_executor,
    },
    system_message="Python 프로그래밍 관련 질문을 하고 코드를 실행할 수 있는 사용자입니다."
)

In [4]:
def start_coding_task(task_description):
    user_proxy.initiate_chat(
        assistant,
        message=f"""다음 작업을 수행하고 결과를 실행해주세요: {task_description}
        
        코드 작성 시 포함할 사항:
        1. 코드 설명 및 주석
        2. 실행 가능한 예제
        3. 에러 처리
        4. 실행 결과 확인
        
        코드 실행 시 주의사항:
        1. 실행 전 코드를 ```python 블록으로 감싸주세요
        2. 실행 결과를 확인해주세요
        3. 필요한 경우 코드를 수정해주세요"""
    )

In [5]:
task = """
    다음 두개 코드를 생성해주세요.
    
    1) 최근 1년간 네이버의 주가를 가져와서 csv로 저장하는 코드
    2) 1)에서 저장된 csv를 읽어서 통계를 내는 코드
"""

start_coding_task(task)

User (to Python_Expert):

다음 작업을 수행하고 결과를 실행해주세요: 
    다음 두개 코드를 생성해주세요.
    
    1) 최근 1년간 네이버의 주가를 가져와서 csv로 저장하는 코드
    2) 1)에서 저장된 csv를 읽어서 통계를 내는 코드

        
        코드 작성 시 포함할 사항:
        1. 코드 설명 및 주석
        2. 실행 가능한 예제
        3. 에러 처리
        4. 실행 결과 확인
        
        코드 실행 시 주의사항:
        1. 실행 전 코드를 ```python 블록으로 감싸주세요
        2. 실행 결과를 확인해주세요
        3. 필요한 경우 코드를 수정해주세요

--------------------------------------------------------------------------------
Python_Expert (to User):

네, 두 개의 코드를 작성하겠습니다. 첫 번째 코드는 최근 1년간 네이버의 주가를 가져와 CSV로 저장하는 코드이고, 두 번째 코드는 저장된 CSV 파일을 읽어서 통계를 내는 코드입니다. 코드는 Python의 `pandas`와 `yfinance` 라이브러리를 사용할 것입니다.

### 1. 최근 1년간 네이버의 주가를 가져와서 CSV로 저장하는 코드

```python
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

def fetch_naver_stock_data():
    try:
        # 현재 날짜 및 1년 전 날짜 계산
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)
        
        # 네이버(NAVER)의 주가 데이터 다운로드
        n